In [19]:
# %%capture
#@title Setup Environment
# Install the latest Tensorflow version.
!pip install -q tensorflow-text
!pip install annoy
!pip install simpleneighbors[annoy]
# !pip install -q nltk
# !pip install -q tqdm

In [2]:
import json
import nltk
import os
import pprint
import random
import simpleneighbors
import urllib
import pandas as pd
import numpy as np
from IPython.display import HTML, display
from tqdm.notebook import tqdm

import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer

In [147]:

nltk.download('punkt')

def load_data(path, sep="##,##"):
    data = pd.read_csv(path,sep='##,##')
    return data

def extract_sentences_from_answer(df):
  all_sentences = []
  for index, row in df.iterrows():
#   for ans in answer_list:
      sentences = nltk.tokenize.sent_tokenize(row['answer'])    
      sentences = sentences + nltk.tokenize.sent_tokenize(row['question'])
      all_sentences.extend(zip(sentences, [row['answer']] * len(sentences)))
  return list(set(all_sentences)) # remove duplicates

def extract_questions(df):
  questions = []
  for index, row in df.iterrows():
    questions.append((row['question'], row['answer']))   
  return list(set(questions))
        

def get_nearest(query_text):
  query_embedding = model.signatures['question_encoder'](tf.constant([query_text]))['outputs'][0]
  search_results = index.nearest(query_embedding, n=num_results)
  return search_results

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [121]:
data = load_data('/home/jupyter/faq.csv')
data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


,question,answer
0,What is NEFT ?,National Electronic Funds Transfer or NEFT is ...
1,Where is mount everest ?,Mount Everest is the highest of the Himalayan ...
2,How do I access the system after being locked ...,You may contact the Member Service Center at 8...
3,When I use Popmoney when will the recipient re...,This depends on the delivery option selected b...
4,Does Coastal charge a fee to receive Foreign W...,While Coastal does not charge a fee for receiv...
5,How do I send a secured message?,Within the Messages tab on the Account Service...
6,"Will recurring transfers continue, even if my ...",Absolutely! Any automatic payments or pending ...
7,Where do I find my mobile settings and change ...,If you would like to do things like change you...
8,Why do I keep getting asked my security questi...,The system monitors trends in member access an...
9,How can I view my credit card account details?,"You may access your credit card history, trans..."


In [136]:
sentences = extract_sentences_from_answer(data)
questions = extract_questions(data)
sent_dict = dict(sentences)

In [9]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3" 
model = hub.load(module_url)
model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f4beaccaed0>

In [137]:
#@title Compute embeddings and build simpleneighbors index
batch_size = 1

encodings = model.signatures['response_encoder'](
  input=tf.constant([sentences[0][0]]),
  context=tf.constant([sentences[0][1]]))
index = simpleneighbors.SimpleNeighbors(
    len(encodings['outputs'][0]), metric='angular')

print('Computing embeddings for %s sentences' % len(sentences))
slices = zip(*(iter(sentences),) * batch_size)
num_batches = int(len(sentences) / batch_size)
for s in tqdm(slices, total=num_batches):
  response_batch = list([r for r, c in s])
  context_batch = list([c for r, c in s])
  encodings = model.signatures['response_encoder'](
                input=tf.constant(response_batch),
                context=tf.constant(context_batch)
              )
  print('response_batch =', response_batch)
  for batch_index, batch in enumerate(response_batch):
    index.add_one(batch, encodings['outputs'][batch_index])

index.build()
print('simpleneighbors index for %s sentences built.' % len(sentences))

Computing embeddings for 39 sentences


response_batch = ['You may contact the Member Service Center at 800-868-4262, or visit your local branch.']
response_batch = ['The pink one cost about 250']
response_batch = ['?This can be found?by tapping the menu icon at the top right of the screen and scrolling down until you see the Solution Center.']
response_batch = ['Often times, it can take up to 3 days after accepting the payment.']
response_batch = ['The Digital Banking mobile app contains most of the same features as the desktop site.']
response_batch = ['You may access your credit card history, transactions, payments, and details within Digital Banking.']
response_batch = ['Absolutely!']
response_batch = ['Any automatic payments or pending transaction made before the card was turned off will continue as normal.']
response_batch = ['How do I send a secured message?']
response_batch = ['When I use Popmoney when will the recipient receive the funds?']
response_batch = ['The system monitors trends in member access and activity 

On retrieval, the question is encoded using the **question_encoder** and the question embedding is used to query the simpleneighbors index.

In [148]:
#@title Retrieve nearest neighbors for a random question from SQuAD
num_results = 10 #@param {type:"slider", min:5, max:40, step:1}

s= get_nearest('funds transfer')
for i in s:
    print(i)
# sent_dict[s[0]]

This depends on the delivery option selected by the sender of the funds.
Does Coastal charge a fee to receive Foreign Wire Transfers of Funds?
National Electronic Funds Transfer or NEFT is an electronic funds transfer system maintained by the Reserve Bank of India.
When I use Popmoney when will the recipient receive the funds?
Will recurring transfers continue, even if my card is turned off?
If the sender has selected Next Day or Express delivery then the recipient will receive the funds 1 business days after accepting the payment.
If a fee is imposed, it will be deducted from the amount of the incoming wire prior to us receiving it and posting it to your account.
Any automatic payments or pending transaction made before the card was turned off will continue as normal.
You may access your credit card history, transactions, payments, and details within Digital Banking.
Often times, it can take up to 3 days after accepting the payment.


{'You may contact the Member Service Center at 800-868-4262, or visit your local branch.': 'You may contact the Member Service Center at 800-868-4262, or visit your local branch.',
 'The pink one cost about 250': 'The pink one cost about 250',
 '?This can be found?by tapping the menu icon at the top right of the screen and scrolling down until you see the Solution Center.': 'If you would like to do things like change your password, change your email address or change your address, you just need to access the Solution Center on the mobile app. ?This can be found?by tapping the menu icon at the top right of the screen and scrolling down until you see the Solution Center.',
 'Often times, it can take up to 3 days after accepting the payment.': 'This depends on the delivery option selected by the sender of the funds. Often times, it can take up to 3 days after accepting the payment. If the sender has selected Next Day or Express delivery then the recipient will receive the funds 1 business